# Bienvenido al Modelo Machine Learning

### como primera instancia se importan las librerias que se van a utilizar

In [62]:
import numpy as pd
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

### luego se abre y se mira el DataFrame ya arreglado para la comodidad del usuario

In [76]:
df = pd.read_csv('df_arreglado.csv')

In [77]:
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,id_collection,name_collection,id_genres,name_genres,id_production,name_production,id_countrie,name_countrie,id_language,name_language
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,"16.0,35.0,10751.0","Animation,Comedy,Family",3.0,Pixar Animation Studios,US,United States of America,en,English
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,NaN,NaN,"12.0,14.0,10751.0","Adventure,Fantasy,Family","559.0,2550.0,10201.0","TriStar Pictures,Teitler Film,Interscope Commu...",US,United States of America,"en,fr","English,Français"
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,119050.0,Grumpy Old Men Collection,"10749.0,35.0","Romance,Comedy","6194.0,19464.0","Warner Bros.,Lancaster Gate",US,United States of America,en,English
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,NaN,NaN,"35.0,18.0,10749.0","Comedy,Drama,Romance",306.0,Twentieth Century Fox Film Corporation,US,United States of America,en,English
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,96871.0,Father of the Bride Collection,35.0,Comedy,"5842.0,9195.0","Sandollar Productions,Touchstone Pictures",US,United States of America,en,English


### algo a tener muy en cuenta es que al DataFrame completo render no lo va a correr ya que pesa muchísimo por lo que se crea la función a fin de limitar el mismo, para que pueda ser utilizado en render arrojando un resultado positivo en la consulta

In [78]:
ml=df.sample(n=1000, random_state=42) # limita el ML

### luego se crean las siguientes variables a fin de configurar el ML con etiquetas de guía

In [79]:
tfidf= TfidfVectorizer(stop_words = 'english') # esta  funcion hace que no se detenga el codigo por algun tipo de error
ml['overview'] = ml['overview'].fillna('') # este codigo rellena los nullos con espacios vacios
tfidf_matrix = tfidf.fit_transform(ml['overview']) # entrena el modelo overwiew
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) #defino una función que obtendrá recomendaciones basadas en el título de la película ingresado por el usuario
ml.drop_duplicates(inplace=True) # se dropean duplicados
ml = ml.reset_index(drop=True) # se resetean los Indices para evitar inconvenientes
indices = pd.Series(ml.index, index=ml['title']) # esta variable convierte los títulos de las películas en índices para obtener una respuesta rapida


In [80]:
tfidf_matrix.shape # te arroja el resultado

(1000, 9976)

### se realizan las siguientes funciones

In [47]:
def recomendaciones(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return ml['title'].iloc[movie_indices] # se hizo este y luego

In [81]:
def recomendaciones(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    recomendaciones=list(ml['title'].iloc[movie_indices]) # se mejoro a este 
    return {'lista recomendada': recomendaciones}

### y para finalizar luego de realizar la función definida se miran algunos títulos de películas a fin de realizar una consulta de recomendación

In [82]:
ml['title'].head()

0       Among Friends
1       Sorry, Haters
2                Gone
3    The Three Marias
4       The Overnight
Name: title, dtype: object

In [83]:
recomendaciones('Gone')

{'lista recomendada': ['Schneider vs. Bax',
  'Give Us the Moon',
  'Personal Effects',
  "The Bells of St. Mary's",
  'Little Sister']}

### siendo este el resultado espero que haya sido intuitivo y facil de leer